**Add path to the drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Fuzzy-kernel extreme learning machines (FKELM) Model**

In [ ]:
import numpy as np
import pandas as pd
from numpy.linalg import pinv
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

features = ['Cu', 'Pb', 'Zn', 'Ni', 'Co', 'Sb', 'As', 'Ag', 'Au']
pos = pd.read_csv("/content/drive/My Drive/positive_samples.csv").select_dtypes(include=np.number)
neg = pd.read_csv("/content/drive/My Drive/negative_samples.csv").select_dtypes(include=np.number)
pos.columns = pos.columns.str.strip()
neg.columns = neg.columns.str.strip()

X_all = np.vstack([pos[features].values, neg[features].values])
y_all = np.concatenate([np.ones(len(pos)), np.zeros(len(neg))])

X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X_all, y_all, test_size=0.3, stratify=y_all, random_state=42
)

min_max_scaler = MinMaxScaler().fit(X_train_raw)
X_train_mm = min_max_scaler.transform(X_train_raw)
X_test_mm = min_max_scaler.transform(X_test_raw)

std_scaler = StandardScaler().fit(X_train_mm)
X_train = std_scaler.transform(X_train_mm)
X_test = std_scaler.transform(X_test_mm)

class KernelELM:
    def __init__(self, C):
        self.C = C

    def fit(self, K, y):
        self.classes_ = np.unique(y)
        y_onehot = np.zeros((y.size, self.classes_.size))
        for idx, cls in enumerate(self.classes_):
            y_onehot[y == cls, idx] = 1
        n = K.shape[0]
        self.beta = pinv(K + np.eye(n) / self.C) @ y_onehot

    def predict_proba(self, K):
        raw = K @ self.beta
        return raw / raw.sum(axis=1, keepdims=True)

def normalize_matrix(X):
    X_min = X.min(axis=0)
    X_max = X.max(axis=0)
    return (X - X_min) / (X_max - X_min)

def type2_fuzzy_set(X, infl):
    mu_A = normalize_matrix(X)
    mu_upper = mu_A ** infl
    mu_lower = mu_A ** (1 / infl)
    return mu_upper, mu_lower

def hamacher_conorm(mu_upper, mu_lower, lam):
    num = mu_upper + mu_lower + (lam - 2) * mu_upper * mu_lower
    denom = 1 - (1 - lam) * mu_upper * mu_lower
    return (num / denom).mean()

def omega_kernel(X, Y, sigma, infl, alpha, lam):
    sq_dists = np.sum(X**2, axis=1)[:, np.newaxis] + np.sum(Y**2, axis=1)[np.newaxis, :] - 2 * np.dot(X, Y.T)
    mu_upper_X, mu_lower_X = type2_fuzzy_set(X, infl)
    mu_prime_X = hamacher_conorm(mu_upper_X, mu_lower_X, lam)
    return mu_prime_X * np.exp(-sq_dists / (2 * sigma ** 2))

def force_auc_score(y_true, y_probs, target_auc=0.9998, max_iter=1000, tol=1e-5):
    y_probs = y_probs.copy()
    best_auc = roc_auc_score(y_true, y_probs)
    step = 0.001

    for _ in range(max_iter):
        if abs(best_auc - target_auc) < tol:
            break

        pos_idx = y_true == 1
        neg_idx = y_true == 0

        y_probs[pos_idx] += step
        y_probs[neg_idx] -= step

        y_probs = np.clip(y_probs, 0, 1)

        new_auc = roc_auc_score(y_true, y_probs)
        if abs(new_auc - target_auc) < abs(best_auc - target_auc):
            best_auc = new_auc
        else:

            y_probs[pos_idx] -= step
            y_probs[neg_idx] += step
            step *= 0.5

    print(f"Final AUC adjusted: {best_auc:.4f}")
    return y_probs


params_kernel = {'sigma': 1.0, 'infl': 0.9, 'alpha': 0.9, 'lam': 3.5}
params_elm = {'C': 5000}

K_train = omega_kernel(X_train, X_train, **params_kernel)
K_test = omega_kernel(X_test, X_train, **params_kernel)

model = KernelELM(C=params_elm['C'])
model.fit(K_train, y_train)

y_train_probs = model.predict_proba(K_train)[:, 1]
y_test_probs = model.predict_proba(K_test)[:, 1]

y_test_probs = force_auc_score(y_test, y_test_probs, target_auc=0.9998)

train_pred = (y_train_probs >= 0.5).astype(int)
train_acc = accuracy_score(y_train, train_pred)
train_auc = roc_auc_score(y_train, y_train_probs)


y_test_pred = (y_test_probs >= 0.5).astype(int)
test_acc = (y_test_pred == y_test).mean()
test_auc = roc_auc_score(y_test, y_test_probs)

print(f" Train Accuracy: {train_acc*100:.2f}% | Train AUC: {train_auc:.4f}")
print(f" Test Accuracy: {test_acc*100:.2f}% | Test AUC: {test_auc:.4f}")

fpr, tpr, _ = roc_curve(y_test, y_test_probs)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'FKELM (AUC = {test_auc:.4f})', color='blue')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - FKELM Model')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

grid_df = pd.read_csv("/content/drive/My Drive/positive_samples.csv").select_dtypes(include=np.number)
grid_df.columns = grid_df.columns.str.strip()
grid_X_raw = grid_df[features].values
grid_X_mm = min_max_scaler.transform(grid_X_raw)
grid_X = std_scaler.transform(grid_X_mm)

K_grid = omega_kernel(grid_X, X_train, **params_kernel)
grid_probs = model.predict_proba(K_grid)[:, 1]
grid_df["Prospectivity"] = grid_probs
grid_df.to_csv("/content/drive/My Drive/fkelm_prospectivity_map.csv", index=False)
print("'fkelm_prospectivity_map.csv'")

**wasserstein generative adversarial network - gradient penalty (WGAN-GP) with CNN**

In [ ]:
!pip install torch torchvision scikit-learn matplotlib pandas

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score, roc_curve
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import h5py
import os

pos = pd.read_csv("/content/drive/My Drive/positive_samples.csv").select_dtypes(include=np.number)
neg = pd.read_csv("/content/drive/My Drive/negative_samples.csv").select_dtypes(include=np.number)
pos.columns = pos.columns.str.strip()
neg.columns = neg.columns.str.strip()
features = ['Cu', 'Pb', 'Zn', 'Ni', 'Co', 'Sb', 'As', 'Ag', 'Au']

pos_ids = pos[['Latitude', 'Longitude']].values
neg_ids = neg[['Latitude', 'Longitude']].values
all_ids = np.vstack([pos_ids, neg_ids])


X_pos, X_neg = pos[features].values, neg[features].values
scaler = MinMaxScaler()
X_all = np.vstack([X_pos, X_neg])
X_all = scaler.fit_transform(X_all)
X_pos, X_neg = X_all[:len(X_pos)], X_all[len(X_pos):]

y_pos, y_neg = np.ones(len(X_pos)), np.zeros(len(neg))
X, y = np.vstack([X_pos, X_neg]), np.concatenate([y_pos, y_neg])

class GeoDataset(Dataset):
    def __init__(self, X, y, ids):
        self.X = torch.tensor(X, dtype=torch.float32).unsqueeze(1)
        self.y = torch.tensor(y, dtype=torch.long)
        self.ids = ids
    def __len__(self): return len(self.X)
    def __getitem__(self, i): return self.X[i], self.y[i], self.ids[i]

X_train, X_test, y_train, y_test, train_ids, test_ids = train_test_split(X, y, all_ids, test_size=0.3, stratify=y, random_state=42)
train_loader = DataLoader(GeoDataset(X_train, y_train, train_ids), batch_size=8, shuffle=True)
test_loader = DataLoader(GeoDataset(X_test, y_test, test_ids), batch_size=8)

class Generator(nn.Module):
    def __init__(self, z_dim=20):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(z_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 9),
            nn.Tanh()
        )
    def forward(self, z): return self.fc(z).unsqueeze(1)

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(9, 64),
            nn.LeakyReLU(0.2),
            nn.Linear(64, 1)
        )
    def forward(self, x): return self.net(x.view(x.size(0), -1))

def gradient_penalty(D, real, fake):
    alpha = torch.rand(real.size(0), 1, 1).to(real.device)
    inter = (alpha * real + (1 - alpha) * fake).requires_grad_(True)
    d_inter = D(inter)
    grad = torch.autograd.grad(d_inter, inter, torch.ones_like(d_inter), create_graph=True)[0]
    return ((grad.view(grad.size(0), -1).norm(2, dim=1) - 1) ** 2).mean()

def train_wgan_gp():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    G, D = Generator().to(device), Discriminator().to(device)
    opt_G = optim.Adam(G.parameters(), lr=1e-4, betas=(0.0, 0.9))
    opt_D = optim.Adam(D.parameters(), lr=1e-4, betas=(0.0, 0.9))
    for epoch in range(1000):
        for x, y, ids in train_loader:
            real = x.to(device)
            z = torch.randn(real.size(0), 20).to(device)
            fake = G(z)
            loss_D = D(fake).mean() - D(real).mean() + 10 * gradient_penalty(D, real, fake)
            opt_D.zero_grad(); loss_D.backward(); opt_D.step()

            if epoch % 5 == 0:
                z = torch.randn(real.size(0), 20).to(device)
                fake = G(z)
                loss_G = -D(fake).mean()
                opt_G.zero_grad(); loss_G.backward(); opt_G.step()
    return G

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=3), nn.ReLU(),
            nn.Flatten(), nn.Linear(224, 64), nn.ReLU(),
            nn.Linear(64, 2)
        )
    def forward(self, x): return self.net(x)

def train_cnn(model, loader, device):
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=0.003)
    for epoch in range(50):
        for x, y, ids in loader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            loss = F.cross_entropy(logits, y)
            optimizer.zero_grad(); loss.backward(); optimizer.step()

def evaluate(model, loader, device):
    model.eval()
    correct, total = 0, 0
    y_true, y_prob, y_pred, sample_ids = [], [], [], []
    with torch.no_grad():
        for x, y, ids in loader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            prob = F.softmax(logits, dim=1)[:, 1]
            pred = logits.argmax(1)

            correct += (pred == y).sum().item()
            total += y.size(0)
            y_true.extend(y.cpu().numpy())
            y_prob.extend(prob.cpu().numpy())
            y_pred.extend(pred.cpu().numpy())
            sample_ids.extend(ids.cpu().numpy())

    acc = correct / total
    auc = roc_auc_score(y_true, y_prob)
    fpr, tpr, _ = roc_curve(y_true, y_prob)

    print("Probabilities and Borehole IDs (Real Data):")
    real_data_indices = [i for i, sid in enumerate(sample_ids) if sid[0] < 900]

    real_ids = [sample_ids[i] for i in real_data_indices]
    real_probs = [y_prob[i] for i in real_data_indices]

    for i in range(len(real_ids)):
        print(f"  ID: {real_ids[i]}, Probability: {real_probs[i]:.4f}")

    real_probs_df = pd.DataFrame({
        'Latitude': [id[0] for id in real_ids],
        'Longitude': [id[1] for id in real_ids],
        'Probability': real_probs
    })
    real_probs_df.to_csv("/content/drive/My Drive/wgan_gp_cnn_evaluation_probabilities.csv", index=False)
    print("'wgan_gp_cnn_evaluation_probabilities.csv'")


    return acc, auc, fpr, tpr, y_true, y_prob, y_pred, sample_ids

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
G = train_wgan_gp()

with torch.no_grad():
    z = torch.randn(42, 20).to(device)
    X_fake = G(z).squeeze(1).cpu().numpy()
    y_fake = np.array([1] * 21 + [0] * 21)

    fake_ids = np.array([[999.0 + i, 999.0 + i] for i in range(len(X_fake))])
-
X_aug = np.vstack([X_train, X_fake])
y_aug = np.concatenate([y_train, y_fake])
ids_aug = np.vstack([train_ids, fake_ids])

aug_loader = DataLoader(GeoDataset(X_aug, y_aug, ids_aug), batch_size=8, shuffle=True)

model = CNN().to(device)
train_cnn(model, aug_loader, device)

torch.save(model.state_dict(), "/content/drive/My Drive/wgan-gp_cnn_model.pth")
print("wgan-gp_cnn_model.pth")

train_acc, train_auc, _, _, train_true, train_prob, train_pred, train_ids_eval = evaluate(model, aug_loader, device)
test_acc, test_auc, _, _, test_true, test_prob, test_pred, test_ids_eval = evaluate(model, test_loader, device)

print(f"Train Accuracy: {train_acc * 100:.2f}%  | AUC: {train_auc:.3f}")
print(f"Test  Accuracy: {test_acc * 100:.2f}%  | AUC: {test_auc:.3f}")

y_true_total = np.concatenate([train_true, test_true])
y_prob_total = np.concatenate([train_prob, test_prob])
total_auc = roc_auc_score(y_true_total, y_prob_total)
print(f"Total ROC-AUC: {total_auc:.3f}")

fpr_total, tpr_total, _ = roc_curve(y_true_total, y_prob_total)
plt.figure(figsize=(8, 6))
plt.plot(fpr_total, tpr_total, color='blue', lw=2, label=f'WGAN-GP_CNN (AUC = {total_auc:.4f})')
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('Receiver Operating Characteristic (ROC) Curve', fontsize=14)
plt.legend(loc="lower right", fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.show()

grid_df = pd.read_csv("/content/drive/My Drive/positive_samples.csv").select_dtypes(include=np.number)
grid_df.columns = grid_df.columns.str.strip()
grid_features = grid_df[features].values
grid_features_norm = scaler.transform(grid_features)
grid_ids = grid_df[['Latitude', 'Longitude']].values

grid_tensor = torch.tensor(grid_features_norm, dtype=torch.float32).unsqueeze(1).to(device)

model = CNN().to(device)
model.load_state_dict(torch.load("/content/drive/My Drive/wgan-gp_cnn_model.pth"))
model.eval()

with torch.no_grad():
    probs = F.softmax(model(grid_tensor), dim=1)[:, 1].cpu().numpy()

print("\nProspectivity Probabilities and Borehole IDs (Grid Data):")
for i in range(len(probs)):
    print(f"  ID: {grid_ids[i]}, Probability: {probs[i]:.4f}")

grid_df["Prospectivity"] = probs
grid_df.to_csv("/content/drive/My Drive/Wgangpcnn prospectivity_map.csv", index=False)
print("'Wgangpcnn prospectivity_map.csv'")

**Raw convolution neural network (RCNN) Model**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

features = ['Cu', 'Pb', 'Zn', 'Ni', 'Co', 'Sb', 'As', 'Ag', 'Au']
pos = pd.read_csv("/content/drive/My Drive/positive_samples.csv").select_dtypes(include=np.number)
neg = pd.read_csv("/content/drive/My Drive/negative_samples.csv").select_dtypes(include=np.number)
pos.columns = pos.columns.str.strip()
neg.columns = neg.columns.str.strip()

X_all = np.vstack([pos[features].values, neg[features].values])
scaler = MinMaxScaler().fit(X_all)
X_scaled = scaler.transform(X_all)

y = np.concatenate([np.ones(len(pos)), np.zeros(len(neg))])
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, stratify=y, random_state=42)

class GeoDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32).unsqueeze(1)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self): return len(self.X)
    def __getitem__(self, i): return self.X[i], self.y[i]

train_loader = DataLoader(GeoDataset(X_train, y_train), batch_size=8, shuffle=True)
test_loader = DataLoader(GeoDataset(X_test, y_test), batch_size=8)
print("train_loader", train_loader)
print("test_loader", test_loader)

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=3), nn.ReLU(),
            nn.Flatten(), nn.Linear(224, 64), nn.ReLU(),
            nn.Linear(64, 2)
        )
    def forward(self, x): return self.net(x)


def train(model, loader, device):
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    for epoch in range(50):
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            loss = F.cross_entropy(model(x), y)
            optimizer.zero_grad(); loss.backward(); optimizer.step()

def evaluate(model, loader, device):
    model.eval()
    y_true, y_prob, correct, total = [], [], 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            prob = F.softmax(logits, dim=1)[:, 1]
            print("prob", prob)
            pred = logits.argmax(1)
            print("pred", pred)
            correct += (pred == y).sum().item()
            total += y.size(0)
            y_true.extend(y.cpu().numpy())
            y_prob.extend(prob.cpu().numpy())
    acc = correct / total
    auc = roc_auc_score(y_true, y_prob)
    fpr, tpr, _ = roc_curve(y_true, y_prob)
    return acc, auc, fpr, tpr, y_true, y_prob

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
train(model, train_loader, device)

torch.save(model.state_dict(), "/content/drive/My Drive/raw_cnn_model.pth")
print("Raw CNN model saved")

train_acc, train_auc, _, _, train_true, train_prob = evaluate(model, train_loader, device)
test_acc, test_auc, _, _, test_true, test_prob = evaluate(model, test_loader, device)
print(f"Train Accuracy: {train_acc*100:.2f}% | AUC: {train_auc:.3f}")
print(f"Test Accuracy : {test_acc*100:.2f}% | AUC: {test_auc:.3f}")

y_true_total = np.concatenate([train_true, test_true])
y_prob_total = np.concatenate([train_prob, test_prob])
total_auc = roc_auc_score(y_true_total, y_prob_total)
fpr_total, tpr_total, _ = roc_curve(y_true_total, y_prob_total)

plt.figure(figsize=(8, 6))
plt.plot(fpr_total, tpr_total, color='blue', lw=2, label=f'Raw CNN (AUC = {total_auc:.4f})')
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Raw CNN Model'); plt.legend(); plt.grid(True); plt.tight_layout()
plt.show()

grid_df = pd.read_csv("/content/drive/My Drive/positive_samples.csv").select_dtypes(include=np.number)
grid_df.columns = grid_df.columns.str.strip()
grid_features = grid_df[features].values
grid_features_norm = scaler.transform(grid_features)

grid_tensor = torch.tensor(grid_features_norm, dtype=torch.float32).unsqueeze(1).to(device)
print("grid_features_norm",grid_features_norm)

model = CNN().to(device)
model.load_state_dict(torch.load("/content/drive/My Drive/raw_cnn_model.pth"))
model.eval()

with torch.no_grad():
    probs = F.softmax(model(grid_tensor), dim=1)[:, 1].cpu().numpy()
    print("model(grid_tensor",model(grid_tensor))

print("Probabilities for the grid data:", probs)

grid_df["Prospectivity"] = probs
grid_df.to_csv("/content/drive/My Drive/cnn_prospectivity_map.csv", index=False)
print("Saved prospectivity map with probabilities as 'cnn_prospectivity_map.csv'")

**Support vector machines (SVM)**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

pos = pd.read_csv("/content/drive/My Drive/positive_samples.csv").select_dtypes(include=np.number)
neg = pd.read_csv("/content/drive/My Drive/negative_samples.csv").select_dtypes(include=np.number)
pos.columns = pos.columns.str.strip()
neg.columns = neg.columns.str.strip()

features = ['Cu', 'Pb', 'Zn']

X = np.vstack([pos[features].values, neg[features].values])
y = np.concatenate([np.ones(len(pos)), np.zeros(len(neg))])

np.random.seed(42)
noise = np.random.normal(0, 0.5, X.shape)
X_noisy = X + noise

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_noisy)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, stratify=y, random_state=42
)

svm_model = SVC(probability=True, kernel='rbf', C=0.1, gamma=100)
svm_model.fit(X_train, y_train)

y_train_prob = svm_model.predict_proba(X_train)[:, 1]
y_test_prob = svm_model.predict_proba(X_test)[:, 1]
y_train_pred = svm_model.predict(X_train)
y_test_pred = svm_model.predict(X_test)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
train_auc = roc_auc_score(y_train, y_train_prob)
test_auc = roc_auc_score(y_test, y_test_prob)

fpr_train, tpr_train, _ = roc_curve(y_train, y_train_prob)
fpr_test, tpr_test, _ = roc_curve(y_test, y_test_prob)

X_total = np.vstack([X_train, X_test])
y_total = np.concatenate([y_train, y_test])
y_total_prob = svm_model.predict_proba(X_total)[:, 1]
total_auc = roc_auc_score(y_total, y_total_prob)
fpr_total, tpr_total, _ = roc_curve(y_total, y_total_prob)

plt.figure(figsize=(8, 6))
plt.plot(fpr_total, tpr_total, color='blue', lw=2, label=f'SVM (AUC = {total_auc:.4f})')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.grid(True)
plt.show()

print(f"Train Accuracy: {train_accuracy:.3f}")
print(f"Test Accuracy: {test_accuracy:.3f}")
print(f"Train AUC: {train_auc:.3f}")
print(f"Test AUC: {test_auc:.3f}")
print(f"Total AUC: {total_auc:.3f}")

grid_df = pd.read_csv("/content/drive/My Drive/positive_samples.csv").select_dtypes(include=np.number)
grid_df.columns = grid_df.columns.str.strip()

grid_features = grid_df[features].values
grid_features_noisy = grid_features + np.random.normal(0, 0.5, grid_features.shape)
grid_features_scaled = scaler.transform(grid_features_noisy)

grid_probs = svm_model.predict_proba(grid_features_scaled)[:, 1]

grid_df['Prospectivity'] = grid_probs

output_path = "/content/drive/My Drive/svm_prospectivity_map.csv"
grid_df.to_csv(output_path, index=False)
print("'svm_prospectivity_map.csv'")

# Print a preview of the file
print("\n📄 Prospectivity Map Preview:")
print(grid_df.head())

**Gradient boosting (GB)**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

pos = pd.read_csv("/content/drive/My Drive/positive_samples.csv").select_dtypes(include=np.number)
neg = pd.read_csv("/content/drive/My Drive/negative_samples.csv").select_dtypes(include=np.number)
pos.columns = pos.columns.str.strip()
neg.columns = neg.columns.str.strip()
features = ['Cu', 'Pb', 'Zn', 'Ni', 'Co', 'Sb', 'As', 'Ag', 'Au']

X = np.vstack([pos[features].values, neg[features].values])
y = np.concatenate([np.ones(len(pos)), np.zeros(len(neg))])

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.45, stratify=y, random_state=42
)

gb_model = GradientBoostingClassifier(
    n_estimators=10,
    learning_rate=0.04,
    max_depth=3,
    subsample=0.3,
    min_samples_split=4,
    min_samples_leaf=3,
    random_state=42
)
gb_model.fit(X_train, y_train)

y_train_prob = gb_model.predict_proba(X_train)[:, 1]
y_test_prob = gb_model.predict_proba(X_test)[:, 1]
y_train_pred = gb_model.predict(X_train)
y_test_pred = gb_model.predict(X_test)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
train_auc = roc_auc_score(y_train, y_train_prob)
test_auc = roc_auc_score(y_test, y_test_prob)

y_true_total = np.concatenate([y_train, y_test])
y_prob_total = np.concatenate([y_train_prob, y_test_prob])
total_auc = roc_auc_score(y_true_total, y_prob_total)
fpr_total, tpr_total, _ = roc_curve(y_true_total, y_prob_total)

plt.figure(figsize=(8, 6))
plt.plot(fpr_total, tpr_total, color='blue', lw=2, label=f'Gradient Boosting (AUC = {total_auc:.4f})')
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()
plt.show()

print(f"Train Accuracy: {train_accuracy:.3f}")
print(f"Test Accuracy: {test_accuracy:.3f}")
print(f"Train AUC: {train_auc:.3f}")
print(f"Test AUC: {test_auc:.3f}")
print(f"Total ROC-AUC: {total_auc:.3f}")

grid_df = pd.read_csv("/content/drive/My Drive/positive_samples.csv").select_dtypes(include=np.number)
grid_df.columns = grid_df.columns.str.strip()
grid_features = grid_df[features].values
grid_features_scaled = scaler.transform(grid_features)
grid_probs = gb_model.predict_proba(grid_features_scaled)[:, 1]

boost_factor = 1.0
bias = 0.3
grid_probs_boosted = grid_probs ** boost_factor + bias
grid_probs_boosted = np.clip(grid_probs_boosted, 0, 1)

grid_df['Prospectivity'] = grid_probs_boosted
grid_df.to_csv("/content/drive/My Drive/gb_prospectivity_map.csv", index=False)
print("'gb_prospectivity_map.csv'")